<a href="https://colab.research.google.com/github/acmilannesta/Pediatric-readmission/blob/master/Pedatric%20readmission%20Catboost%20%26%20LGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!git clone https://github.com/acmilannesta/Pediatric-readmission

fatal: destination path 'Pediatric-readmission' already exists and is not an empty directory.


In [4]:
!pip install catboost

     |████████████████████████████████| 61.1MB 1.2MB/s 


In [0]:
import pandas as pd, numpy as np, gc
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
from hyperopt import fmin, hp, tpe, STATUS_OK, Trials
from hyperopt.pyll import scope

In [0]:
one = pd.read_sas('Pediatric-readmission/peds_usrds_validation.sas7bdat', encoding='latin1')
two = pd.read_sas('Pediatric-readmission/extra_variables_06012019.sas7bdat', encoding='latin1')
three = one.merge(two, on='MRN')
features_list = [
            'COMO_ALCHO',
            'COMO_CHF',
            'COMO_COPD',
            'COMO_CVATIA',
            'COMO_DRUG',
            'COMO_INAMB',
            'COMO_INTRANS',
            'COMO_TOBAC',
            'DAGE',
            'DRACE',
            'DSEX',
            'DTYPE',
            'MATC',
#             'PDIS',
            'RACE',
            'RAGE',
            'RSEX',
            'RXDETAIL',
            'Thymoglobulin',
            'anti_il2',
            'cmv_risk',
            'dbmi',
            'diabetes',
            'induction',
            'los',
            'modality_tx',
            'prior_tx',
            'rbmi_tx',
            'rtx',
            'steriods',
            'timeto_waitlist',
            'vintage',
            'zipcode_change',
            'cold_isch_pump_ki'
            ]
features_list.extend(two.drop(['MRN','hba1c_bf_discharge'], 1).columns)
X = three[features_list]
y = three['readmit_90day']

# Sum up comorbidities
como = X.filter(regex='COMO_').dropna().apply(lambda x: LabelEncoder().fit_transform(x), 0)
como['como_tot'] = (como.sum(axis=1) > 0).astype(int)
X = X.filter(regex='^(?!COMO_)').join(como['como_tot'])

# Race match
X['RACE_match'] = X['DRACE'].astype(str)+X['RACE'].astype(str)

# Donor and recipient age difference
# Mean: 16.6, Std: 11.3, Median: 15, IQR: 7-26, Min: 0, Max: 57
X['AGE_DIFF'] = (X['DAGE'] - X['RAGE']).abs()

# add polynomial terms to X with high importance
X['timeto_waitlist_p2'] = X['timeto_waitlist'] ** 2
X['timeto_waitlist_p3'] = X['timeto_waitlist'] ** 3
X['rbmi_tx_p2'] = X['rbmi_tx'] ** 2
X['rbmi_tx_p3'] = X['rbmi_tx'] ** 3
X['dbmi_p2'] = X['dbmi'] ** 2
X['dbmi_p3'] = X['dbmi'] ** 3
X['cold_isch_pump_ki_p2'] = X['cold_isch_pump_ki'] ** 2
X['cold_isch_pump_ki_p3'] = X['cold_isch_pump_ki'] ** 3
X['los_p2'] = X['los'] ** 2
X['los_p3'] = X['los'] ** 3

##Catboost Model

In [0]:
X_cat = X.copy()
cat_colidx = [X_cat.columns.get_loc(col) for col in X_cat.columns if X_cat[col].nunique() <= 10 or col in ['RACE_match']]
for col in cat_colidx:
    if X_cat[X_cat.columns[col]].dtype == 'float64':
        X_cat[X_cat.columns[col]] = X_cat[X_cat.columns[col]].fillna(-1).astype(int)
    else:
        X_cat[X_cat.columns[col]] =X_cat[X_cat.columns[col]].fillna('')

In [10]:
cbc_params = {
    'max_depth': scope.int(hp.quniform('max_depth', 2, 11, 1)),
    'l2_leaf_reg': hp.uniform('l2_leaf_reg', 500, 2000),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.05, 0.3),
#     'subsample': hp.uniform('subsample', 0.1, 1),
    'eta': hp.uniform('eta', 0.01, 0.1),
#     'bootstrap_type': hp.choice('bootstrap_type', ['Bernoulli', 'Poisson', 'No']),
#     'one_hot_max_size': scope.int(hp.quniform('one_hot_max_size', 2, 11, 1)),
}

def f_cbc(params):
    kfold = StratifiedKFold(5, True, 2019)
#     auc = np.zeros(kfold.get_n_splits())
    cbc_pred = np.zeros(len(X_cat))
    featureimp = np.zeros(X_cat.shape[1])
    cbc = CatBoostClassifier(
        **params,
        n_estimators=999999,
        random_state=2019,
        eval_metric='AUC',
        cat_features=cat_colidx,
        silent=True,
        one_hot_max_size=2,
#         bootstrap_type='Bernoulli',
#         boosting_type='Plain',
#         task_type='GPU',
    )
    for i, (tr_idx, val_idx) in enumerate(kfold.split(X_cat, y)):
        clf = cbc.fit(X_cat.iloc[tr_idx],
                      y[tr_idx],
                      use_best_model=True,
                      eval_set=(X_cat.iloc[val_idx], y[val_idx]),
                      early_stopping_rounds=500,
                      verbose_eval=False)
        cbc_pred[val_idx] = clf.predict_proba(X_cat.iloc[val_idx])[:, 1]
        featureimp += np.asarray(clf.get_feature_importance()) / kfold.n_splits
#         auc[i] = roc_auc_score(y[val_idx], cbc_pred[val_idx])
        del clf
        gc.collect()
    auc = roc_auc_score(y, cbc_pred)
    return {'loss': -round(auc, 5), 'status': STATUS_OK, 'featureimp': featureimp}

trials = Trials()
cbc_best = fmin(f_cbc, cbc_params, algo=tpe.suggest, rstate=np.random.RandomState(9012), max_evals=50, trials=trials)

100%|██████████| 50/50 [1:39:37<00:00, 123.19s/it, best loss: -0.63402]


In [0]:
print(cbc_best)

{'colsample_bylevel': 0.1768121090550874, 'eta': 0.0400933533641472, 'l2_leaf_reg': 1414.690117033347, 'max_depth': 5.0}


In [0]:
 pd.DataFrame({'features': X_cat.columns, 'importance': trials.best_trial['result']['featureimp']}).sort_values('importance', ascending=False).head(20)

,features,importance
22,vintage,11.502837
37,dbp_stay_mean,6.734919
46,timeto_waitlist_p2,5.553072
2,DSEX,4.985403
6,RAGE,4.652250
20,steriods,4.633234
21,timeto_waitlist,4.229344
54,los_p2,3.994615
34,height_tx,3.814054
12,dbmi,2.941777


##LGBM model

In [11]:
!pip uninstall lightgbm
!pip install lightgbm --install-option=--gpu

Uninstalling lightgbm-2.2.3:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/lightgbm-2.2.3.dist-info/*
    /usr/local/lib/python3.6/dist-packages/lightgbm/*
Proceed (y/n)? y
  Successfully uninstalled lightgbm-2.2.3
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:244: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
     |████████████████████████████████| 655kB 44.8MB/s 
Skipping bdist_wheel for lightgbm, due to binaries being disabled for it.
  Running setup.py install for lightgbm ... done


In [0]:
import os,  pandas as pd,  numpy as np, gc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from hyperopt import fmin, hp, tpe, STATUS_OK, Trials
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgbm
import warnings
warnings.filterwarnings("ignore")

In [0]:
X_lgb = X.copy()
cat_col = [X_lgb.columns.get_loc(col) for col in X_lgb.columns if X_lgb[col].nunique() <= 10]
for col in X_lgb.columns:
    if X_lgb[col].dtype == 'O':
        X_lgb[col] = LabelEncoder().fit_transform(X_lgb[col].fillna('Unknown'))
    elif X_lgb[col].nunique() <= 10:
        X_lgb[col] = LabelEncoder().fit_transform(X_lgb[col].fillna(99))

In [14]:
lgbm_param = {
        'num_leaves': scope.int(hp.quniform('num_leaves', 2, 21, 1)),
        'learning_rate': hp.uniform('learning_rate', 0.005, 0.1),
        'feature_fraction': hp.uniform('feature_fraction', 0.05, 1),
        'max_depth': scope.int(hp.quniform('max_depth', 2, 11, 1)),
        'objective': 'binary',
#         'boosting_type': 'dart',
        'metric': 'auc',
        'verbose': -1,
       'device_type': 'gpu'
    }

def f_lgbm(params):
    lgbm_pred = np.zeros((len(X_lgb), ))
#     auc = np.zeros(5)
    featureimp = np.zeros(X_lgb.shape[1])
    for i, (tr_idx, te_idx) in enumerate(StratifiedKFold(5, True, 2019).split(X_lgb, y)):
        tr_data = lgbm.Dataset(X_lgb.values[tr_idx], y.ravel()[tr_idx], categorical_feature=cat_col)
        te_data = lgbm.Dataset(X_lgb.values[te_idx], y.ravel()[te_idx], categorical_feature=cat_col)
        clf = lgbm.train(params,
                         tr_data,
                         num_boost_round=9999999,
                         verbose_eval=False,
                         valid_sets=[tr_data, te_data],
                         early_stopping_rounds=200,
                    )
        lgbm_pred[te_idx] = clf.predict(X_lgb.values[te_idx], num_iteration=clf.best_iteration)
        featureimp += clf.feature_importance() / 5
#         auc[i] = roc_auc_score(y.ravel()[te_idx], lgbm_pred[te_idx])
        del clf
        gc.collect()
    auc = roc_auc_score(y, lgbm_pred)
    return {'loss': -round(auc, 5), 'status': STATUS_OK, 'featureimp': featureimp}

trials_lgb = Trials()
lgbm_best = fmin(f_lgbm, lgbm_param, algo=tpe.suggest, rstate=np.random.RandomState(2019), max_evals=100, trials=trials_lgb)

100%|██████████| 100/100 [04:49<00:00,  2.65s/it, best loss: -0.73591]


In [15]:
print(lgbm_best)

{'feature_fraction': 0.07723307115424309, 'learning_rate': 0.06553679514969062, 'max_depth': 3.0, 'num_leaves': 21.0}


In [16]:
 pd.DataFrame({'features': X_lgb.columns, 'importance': trials_lgb.best_trial['result']['featureimp']}).sort_values('importance', ascending=False).head(20)

,features,importance
28,hemoglobin_discharge,7.0
21,timeto_waitlist,6.4
50,dbmi_p2,6.4
22,vintage,5.0
33,height_tx,4.2
37,dbp_stay_median,4.0
35,sbp_stay_median,3.8
48,rbmi_tx_p2,3.4
36,dbp_stay_mean,3.4
39,dbp_pct_over_95th,2.8
